# VideoLingo Unified Cloud Server

Combines WhisperX (ASR) + Demucs (Vocal Separation)

**Steps:**
1. Set GPU: Runtime -> Change runtime type -> GPU
2. Paste ngrok token in Step 5
3. Run All (Step 5 will block and keep running)
4. Press Ctrl+C in Step 5 output to stop

In [ ]:
# Step 1: Install Mamba and create environment
import os, sys

ENV_PATH = '/content/conda-envs/videolingo' if 'google.colab' in sys.modules else '/kaggle/working/conda-envs/videolingo' if os.path.exists('/kaggle') else os.path.expanduser('~/conda-envs/videolingo')

print("Installing Mamba...")
if not os.path.exists(os.path.expanduser('~/miniforge3/bin/mamba')):
    !wget -q -O /tmp/miniforge.sh https://github.com/conda-forge/miniforge/releases/latest/download/Miniforge3-Linux-x86_64.sh
    !bash /tmp/miniforge.sh -b -p ~/miniforge3

MAMBA = os.path.expanduser('~/miniforge3/bin/mamba')

print(f"Creating env: {ENV_PATH}")
if os.path.exists(ENV_PATH):
    !{MAMBA} remove -p {ENV_PATH} --all -y -q

!{MAMBA} create -p {ENV_PATH} -c conda-forge -y python=3.10 ffmpeg git pip

# Save config for next steps
import json
with open('.conda_python_path', 'w') as f:
    json.dump({'python_path': f'{ENV_PATH}/bin/python', 'env_path': ENV_PATH}, f)

print("✅ Env created!")
print(f"Python path: {ENV_PATH}/bin/python")

In [ ]:
# Step 2: Activate environment and install dependencies
import json, os, sys

# Load env path
with open('.conda_python_path', 'r') as f:
    cfg = json.load(f)
    ENV_PATH = cfg['env_path']
    PYTHON = cfg['python_path']

# Verify Python path and version
print(f"Python path: {PYTHON}")
!{PYTHON} --version
print(f"\nInstalling dependencies...\n")

# Install PyTorch with CUDA
!{PYTHON} -m pip install --no-input torch==2.0.0 torchaudio==2.0.0 --index-url https://download.pytorch.org/whl/cu118

# Install other dependencies
!{PYTHON} -m pip install --no-input whisperx demucs fastapi uvicorn python-multipart pyngrok

# Install ngrok binary
!{PYTHON} -m pyngrok install

print("\n✅ All dependencies installed!")

In [ ]:
# Step 3: Verify installation
import json

# Load from file
with open('.conda_python_path', 'r') as f:
    cfg = json.load(f)
    PYTHON = cfg['python_path']

# Verify Python path and version
print(f"Python path: {PYTHON}")
!{PYTHON} --version
print(f"\nVerifying packages...")

packages = ['torch', 'whisperx', 'demucs', 'fastapi']
for pkg in packages:
    result = !{PYTHON} -c "import {pkg}; print('OK')" 2>&1
    status = '✅' if result and result[0] == 'OK' else '❌'
    print(f"  {status} {pkg}")

# Check GPU
print(f"\nGPU Info:")
!{PYTHON} -c "import torch; print('GPU:', torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU')"

In [ ]:
# Step 4: Download unified server
import urllib.request
import os
import time

# Add timestamp to bypass GitHub cache
timestamp = int(time.time())
url = f"https://raw.githubusercontent.com/infinite-gaming-studio/VideoLingo/main/whisperx_cloud/unified_server.py?t={timestamp}"

# Always delete old file first
if os.path.exists('unified_server.py'):
    os.remove('unified_server.py')
    print("Removed old server file")

# Download new server file
urllib.request.urlretrieve(url, 'unified_server.py')
print("Downloaded unified server")

# Verify version
with open('unified_server.py', 'r', encoding='utf-8') as f:
    content = f.read()
    if 'SERVER_VERSION' in content:
        for line in content.split('\n'):
            if 'SERVER_VERSION' in line and '=' in line and not line.strip().startswith('#'):
                version = line.split('=')[1].strip().replace('"', "").replace("'", '')
                print(f"Server version: {version}")
                break

In [ ]:
# Step 5: Start server (paste your ngrok token below)
NGROK_TOKEN = ""

if not NGROK_TOKEN:
    raise ValueError('Please paste ngrok token')

import json
with open('.conda_python_path', 'r') as f:
    cfg = json.load(f)
    PYTHON = cfg['python_path']

# Start server
!{PYTHON} -c "
import sys
sys.path.insert(0, '/content')
from pyngrok import ngrok, conf
conf.get_default().auth_token = '{NGROK_TOKEN}'
try:
    ngrok.kill()
except:
    pass
url = ngrok.connect(8000, 'http')
print('URL:', url)
print('config.yaml:\n  whisper:\n    runtime: cloud\n    whisperX_cloud_url:', url, '\n  demucs: cloud')
import unified_server
"